In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pydts.examples_utils.generate_simulations_data import generate_quick_start_df

# Data Generation
real_coef_dict = {
    "alpha": {
        1: lambda t: -1 - 0.3 * np.log(t),
        2: lambda t: -1.75 - 0.15 * np.log(t)},
    "beta": {
        1: -np.log([0.8, 3, 3, 2.5, 2]),
        2: -np.log([1, 3, 4, 3, 2])}}

patients_df = generate_quick_start_df(n_patients=50000, n_cov=5, d_times=30, j_events=2, pid_col='pid', 
                                      seed=0, censoring_prob=0.8, real_coef_dict=real_coef_dict)

train_df, test_df = train_test_split(patients_df, test_size=0.2)

# DataExpansionFitter Usage
from pydts.fitters import DataExpansionFitter
fitter = DataExpansionFitter()
fitter.fit(df=train_df.drop(['C', 'T'], axis=1))

pred_df = fitter.predict_cumulative_incident_function(test_df.drop(['J', 'T', 'C', 'X'], axis=1))

# TwoStagesFitter Usage
from pydts.fitters import TwoStagesFitter
new_fitter = TwoStagesFitter()
new_fitter.fit(df=train_df.drop(['C', 'T'], axis=1))

pred_df = fitter.predict_cumulative_incident_function(test_df.drop(['J', 'T', 'C', 'X'], axis=1))

# Training with Regularization
L1_regularized_fitter = TwoStagesFitter()
fit_beta_kwargs = {'model_kwargs': {'penalizer': 0.003, 'l1_ratio': 1}}
L1_regularized_fitter.fit(df=train_df.drop(['C', 'T'], axis=1), fit_beta_kwargs=fit_beta_kwargs)

L2_regularized_fitter = TwoStagesFitter()
fit_beta_kwargs = {'model_kwargs': {'penalizer': 0.003, 'l1_ratio': 0}}
L2_regularized_fitter.fit(df=train_df.drop(['C', 'T'], axis=1), fit_beta_kwargs=fit_beta_kwargs)

EN_regularized_fitter = TwoStagesFitter()
fit_beta_kwargs = {'model_kwargs': {'penalizer': 0.003, 'l1_ratio': 0.5}}
EN_regularized_fitter.fit(df=train_df.drop(['C', 'T'], axis=1), fit_beta_kwargs=fit_beta_kwargs)